***GENERATED CODE FOR forecastingpropertyvaluesregression1 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date Recorded', 'transformation_label': 'String Indexer'}], 'feature': 'Date Recorded', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '1/10/2002', 'max': '9/9/2021', 'missing': '0', 'distinct': '228'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date Recorded'}, {'feature_label': 'Date Recorded', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date Recorded')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Town', 'transformation_label': 'String Indexer'}], 'feature': 'Town', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Ansonia/Massachusetts', 'max': 'Woodstock/New York', 'missing': '0', 'distinct': '86'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Town'}, {'feature_label': 'Town', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Town')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Address', 'transformation_label': 'String Indexer'}], 'feature': 'Address', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 FAWN RDG', 'max': 'Windham/New York ROAD', 'missing': '0', 'distinct': '499'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Address'}, {'feature_label': 'Address', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Address')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sale Amount', 'threshold': 411307.858, 'transformation_label': 'Binarizer'}], 'feature': 'Sale Amount', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '411307.86', 'stddev': '972429.18', 'min': '1080.0', 'max': '1.8E7', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sale Amount'}, {'feature_label': 'Sale Amount', 'threshold': 411307.858, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Sale Amount')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Property Type', 'transformation_label': 'String Indexer'}], 'feature': 'Property Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '446', 'mean': '', 'stddev': '', 'min': 'Condo', 'max': 'Two Family', 'missing': '19', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Property Type'}, {'feature_label': 'Property Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Property Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Residential Type', 'transformation_label': 'String Indexer'}], 'feature': 'Residential Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '446', 'mean': '', 'stddev': '', 'min': 'Condo', 'max': 'Two Family', 'missing': '54', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Residential Type'}, {'feature_label': 'Residential Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Residential Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Non Use Code', 'transformation_label': 'String Indexer'}], 'feature': 'Non Use Code', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '145', 'mean': '', 'stddev': '', 'min': '01 - Family', 'max': '28 - Use Assessment', 'missing': '369', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Non Use Code'}, {'feature_label': 'Non Use Code', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Non Use Code')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Assessor Remarks', 'transformation_label': 'String Indexer'}], 'feature': 'Assessor Remarks', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '43', 'mean': '', 'stddev': '', 'min': '1 0F 3 BUILDINGS SOLD', 'max': 'SOLD TO RENTER', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Assessor Remarks'}, {'feature_label': 'Assessor Remarks', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Assessor Remarks')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OPM remarks', 'transformation_label': 'String Indexer'}], 'feature': 'OPM remarks', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '8', 'mean': '', 'stddev': '', 'min': 'COMPLETELY REHABBED PER MLS INCLUDING NEW REAR DECK', 'max': 'TOTAL RENOVATION PER MLS - SEE PREVIOUS SALE #200023', 'missing': '492', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OPM remarks'}, {'feature_label': 'OPM remarks', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OPM remarks')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location', 'transformation_label': 'String Indexer'}], 'feature': 'Location', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '85', 'mean': '', 'stddev': '', 'min': 'POINT (-71.91421 41.92909)', 'max': 'POINT (-73.64278 41.01639)', 'missing': '409', 'distinct': '108'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location'}, {'feature_label': 'Location', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run forecastingpropertyvaluesregression1Hooks.ipynb
try:
	#sourcePreExecutionHook()

	realestatesales = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/RealEstate Sales.csv', 'filename': 'RealEstate Sales.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(realestatesales)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run forecastingpropertyvaluesregression1Hooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(realestatesales,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Serial Number", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "66024.49", "stddev": "4954.89", "min": "60004", "max": "71784", "missing": "0"}, "updatedLabel": "Serial Number"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "List Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2004.0", "stddev": "0.0", "min": "2004", "max": "2004", "missing": "0"}, "updatedLabel": "List Year"}, {"transformationsData": [{"feature_label": "Date Recorded", "transformation_label": "String Indexer"}], "feature": "Date Recorded", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/10/2002", "max": "9/9/2021", "missing": "0", "distinct": "228"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date Recorded"}, {"transformationsData": [{"feature_label": "Town", "transformation_label": "String Indexer"}], "feature": "Town", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Ansonia/Massachusetts", "max": "Woodstock/New York", "missing": "0", "distinct": "86"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Town"}, {"transformationsData": [{"feature_label": "Address", "transformation_label": "String Indexer"}], "feature": "Address", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 FAWN RDG", "max": "Windham/New York ROAD", "missing": "0", "distinct": "499"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Address"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Assessed Value", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "239781.02", "stddev": "310385.84", "min": "1100", "max": "2712290", "missing": "0"}, "updatedLabel": "Assessed Value"}, {"transformationsData": [{"feature_label": "Sale Amount", "threshold": 411307.858, "transformation_label": "Binarizer"}], "feature": "Sale Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "411307.86", "stddev": "972429.18", "min": "1080.0", "max": "1.8E7", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Sale Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales Ratio", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.33", "stddev": "6.98", "min": "0.0", "max": "120.3901754", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales Ratio"}, {"transformationsData": [{"feature_label": "Property Type", "transformation_label": "String Indexer"}], "feature": "Property Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "446", "mean": "", "stddev": "", "min": "Condo", "max": "Two Family", "missing": "19", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Property Type"}, {"transformationsData": [{"feature_label": "Residential Type", "transformation_label": "String Indexer"}], "feature": "Residential Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "446", "mean": "", "stddev": "", "min": "Condo", "max": "Two Family", "missing": "54", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Residential Type"}, {"transformationsData": [{"feature_label": "Non Use Code", "transformation_label": "String Indexer"}], "feature": "Non Use Code", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "145", "mean": "", "stddev": "", "min": "01 - Family", "max": "28 - Use Assessment", "missing": "369", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Non Use Code"}, {"transformationsData": [{"feature_label": "Assessor Remarks", "transformation_label": "String Indexer"}], "feature": "Assessor Remarks", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "43", "mean": "", "stddev": "", "min": "1 0F 3 BUILDINGS SOLD", "max": "SOLD TO RENTER", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Assessor Remarks"}, {"transformationsData": [{"feature_label": "OPM remarks", "transformation_label": "String Indexer"}], "feature": "OPM remarks", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "8", "mean": "", "stddev": "", "min": "COMPLETELY REHABBED PER MLS INCLUDING NEW REAR DECK", "max": "TOTAL RENOVATION PER MLS - SEE PREVIOUS SALE #200023", "missing": "492", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OPM remarks"}, {"transformationsData": [{"feature_label": "Location", "transformation_label": "String Indexer"}], "feature": "Location", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "85", "mean": "", "stddev": "", "min": "POINT (-71.91421 41.92909)", "max": "POINT (-73.64278 41.01639)", "missing": "409", "distinct": "108"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run forecastingpropertyvaluesregression1Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["Serial Number", "List Year", "Assessed Value", "Sales Ratio", "Date Recorded_stringindexer", "Town_stringindexer", "Address_stringindexer", "Property Type_stringindexer", "Residential Type_stringindexer", "Non Use Code_stringindexer", "Assessor Remarks_stringindexer", "OPM remarks_stringindexer", "Location_stringindexer"], "Sale Amount_binarizer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

